# Задание 3 — деобезличивание (анализ + воспроизводимое решение)

Цель ноутбука:
1. Изучить предоставленный файл и понять, **каким способом** обезличены поля.
2. Восстановить обезличенные данные

In [1]:
import re
import os
import pandas as pd
import numpy as np
from typing import List

## 1. Подготовка данных

In [2]:
INPUT_PATH = "../data/task-3/Задание-3-данные.xlsx"
raw = pd.read_excel(INPUT_PATH)
raw.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,Телефон,email,Адрес
1,NaN,47f0896ebfa7f70683e82c26bfdba33178d1d6a8,xgdlt@bpgfjpgsi.qxo,въ. Псчпщэьабявщбэяп Ычщэоьп у.5 щс.476
2,NaN,329510138dad2c8ae131b6c55a60ff4dbed1a43a,eyzuqky@nuzsgor.ius,Тлмкщужцфкуже щс. к.42 ри.379
3,NaN,9da60383009e86d30c873f3f2b0a928a34575274,zivlg38@ouiqt.kwu,Ымияхиз ыу. м.44 тк.188
4,NaN,94fd3b56f2fec4c694465c84af9219e3c11d9ab2,erewxewme55@kqemp.gsq,уп. Хпджя и.34 ож.468
5,NaN,2193b62bcfa2910745a5d80602d721dcc354361a,szgo.dcifcg@acgqwgyw.qca,Юьфтуяаруыяшон бщ. т.73 шр.174
6,NaN,9fd31a1db8d4177b8f624f7642634c515cba9bfd,dbowlvki.toppoboi@rydwksv.myw,эх. Йъапмыфтп Щшхйче-2 о.67 фм.470
7,NaN,e742b4c0758720cd6e76e87d36dfd58565c18702,mkxot.mnkvhmmx@chgxl.hkz,Вгыцбщут жю. ч.95 эх.462
8,NaN,ad71f14a8bc97e653a4648423d1028a0e4036d7c,lhlk.ljbo@elqjxfi.zlj,Юдчгьдибчц кв. ы.6 бщ.271
9,NaN,8d99c58c8f9e4a422f0a74b5528abdcfee62893b,hpmeofs.mjob@mvcpxjua.dpn,Ржсжатмбгтлйк ржс. е.80 лг.247


### 1.1) Приведение к нормализованной таблице

В файле наблюдается строка, где "заголовки" лежат как данные, а дальше строки с самими значениями.

Нормализуем в `df` с колонками: `Телефон`, `email`, `Адрес`.

In [3]:
raw.columns = ["_c0", "Телефон", "email", "Адрес"]
raw = raw.iloc[1:].reset_index(drop=True)    
df = raw[["Телефон", "email", "Адрес"]].copy()
df.head()

,Телефон,email,Адрес
0,47f0896ebfa7f70683e82c26bfdba33178d1d6a8,xgdlt@bpgfjpgsi.qxo,въ. Псчпщэьабявщбэяп Ычщэоьп у.5 щс.476
1,329510138dad2c8ae131b6c55a60ff4dbed1a43a,eyzuqky@nuzsgor.ius,Тлмкщужцфкуже щс. к.42 ри.379
2,9da60383009e86d30c873f3f2b0a928a34575274,zivlg38@ouiqt.kwu,Ымияхиз ыу. м.44 тк.188
3,94fd3b56f2fec4c694465c84af9219e3c11d9ab2,erewxewme55@kqemp.gsq,уп. Хпджя и.34 ож.468
4,2193b62bcfa2910745a5d80602d721dcc354361a,szgo.dcifcg@acgqwgyw.qca,Юьфтуяаруыяшон бщ. т.73 шр.174


Проверим размер и несколько строк:

In [4]:
print("Shape: ", df.shape)

Shape:  (19, 3)


## 2) Гипотеза: обратимое обезличивание `email` и `Адрес` — шифр Цезаря (циклический сдвиг алфавита)

Почему возникает такая гипотеза:
- В адресах часто встречаются шаблоны вида `... щс. ...` или `... у. ...` (похоже на `ул.` / `пр.` / `пер.` после замены букв).
- Длина строк, структура (точки, пробелы, цифры) сохраняется — меняются именно **буквы**.
- Это типично для простого шифрования подстановкой.

Проверим это формально:
- задаём алфавиты,
- пробуем разные сдвиги,
- ищем такие сдвиги, при которых появляются типичные "якоря" адресов (`ул.`, `пер.`, `пр.`, `д.`, `кв.`),
- для email дополнительно проверяем, становится ли строка похожей на валидный email.

### 2.1) Алфавиты и функция "сдвиг Цезаря"

Для адресов логично использовать русский алфавит **без "ё"** (упрощённая версия):

`абвгдежзийклмнопрстуфхцчшщъыьэюя` (32 символа)

Для email — латиница `a..z` (26 символов).

**Дешифрование** для сдвига `k`: буква заменяется на букву на `k` позиций "назад" по циклу.

In [5]:
from src.caesar import caesar_shift, dec_addr, dec_email

RUS_ALPHA = "абвгдежзийклмнопрстуфхцчшщъыьэюя"  # без "ё"
ENG_ALPHA = "abcdefghijklmnopqrstuvwxyz"


## 3) Анализ адресов: как автоматически выбрать правильный сдвиг

Идея:
- перебираем `k = 0..31`,
- дешифруем адрес,
- оцениваем "правдоподобие" по наличию типичных токенов адреса (`ул.`, `пер.`, `пр.`, `пл.`, `д.`, `кв.`),
- выбираем `k` с максимальным score.

In [6]:
from src.scoring import score_addr, best_k_by_addr

ADDR_TOKENS = ["ул.", "пер.", "пр.", "пл.", "наб.", "кв.", "д.", "дом", "корп", "стр"]

# Пример: ТОП-3 кандидата для первой строки
example_addr = df.loc[0, "Адрес"]
cands = [(k, score_addr(dec_addr(example_addr, k)), dec_addr(example_addr, k)) for k in range(len(RUS_ALPHA))]
sorted(cands, key=lambda x: x[1], reverse=True)[:3]

[(15, 16, 'ул. Авиаконструктора Микояна д.5 кв.476'),
 (6, 3, 'ьф. Йлсйучцъыщьуычщй Хсучицй н.5 ул.476'),
 (13, 3, 'хн. Вдквмрпуфтхмфртв Окмрбпв ж.5 мд.476')]

Теперь найдём лучший `k` для всех строк по адресам:

In [7]:
addr_keys = []
addr_scores = []
for a in df["Адрес"].astype(str):
    k, sc = best_k_by_addr(a)
    addr_keys.append(k)
    addr_scores.append(sc)

df_addr = df.copy()
df_addr["k_addr"] = addr_keys
df_addr["addr_score"] = addr_scores
df_addr.head()

,Телефон,email,Адрес,k_addr,addr_score
0,47f0896ebfa7f70683e82c26bfdba33178d1d6a8,xgdlt@bpgfjpgsi.qxo,въ. Псчпщэьабявщбэяп Ычщэоьп у.5 щс.476,15,16
1,329510138dad2c8ae131b6c55a60ff4dbed1a43a,eyzuqky@nuzsgor.ius,Тлмкщужцфкуже щс. к.42 ри.379,6,13
2,9da60383009e86d30c873f3f2b0a928a34575274,zivlg38@ouiqt.kwu,Ымияхиз ыу. м.44 тк.188,8,13
3,94fd3b56f2fec4c694465c84af9219e3c11d9ab2,erewxewme55@kqemp.gsq,уп. Хпджя и.34 ож.468,4,11
4,2193b62bcfa2910745a5d80602d721dcc354361a,szgo.dcifcg@acgqwgyw.qca,Юьфтуяаруыяшон бщ. т.73 шр.174,14,13


Посмотрим распределение ключей по адресам:

In [8]:
df_addr["k_addr"].value_counts().sort_index()

k_addr
1     1
3     1
4     1
6     2
8     1
9     1
10    2
11    1
14    1
15    1
17    1
19    2
20    1
22    2
23    1
Name: count, dtype: int64

Проверим визуально несколько адресов 'до/после':

In [9]:
view = df_addr.sample(7, random_state=0).copy()
view["Адрес_деобезличен"] = [dec_addr(a, k) for a, k in zip(view["Адрес"], view["k_addr"])]
view[["Адрес", "Адрес_деобезличен", "k_addr", "addr_score"]]

,Адрес,Адрес_деобезличен,k_addr,addr_score
10,щс. Торсщыф-Тжрсже к.33 ри.137,ул. Миклухо-Маклая д.33 кв.137,6,13
1,Тлмкщужцфкуже щс. к.42 ри.379,Международная ул. д.42 кв.379,6,13
8,Ржсжатмбгтлйк ржс. е.80 лг.247,Переяславский пер. д.80 кв.247,1,13
18,Аяхвявцювыщъ ацб. х.12 ыу.490,Подсосенский пер. д.12 кв.490,17,13
14,2-в Оифрсувзфнгв цо. з.88 не.299,2-я Леснорядская ул. д.88 кв.299,3,13
16,йб. Щжювцй ъ.34 аш.173,ул. Гримау д.34 кв.173,22,13
6,Вгыцбщут жю. ч.95 эх.462,Пригожая ул. д.95 кв.462,19,13


## 4) Анализ email: подтверждаем тот же подход (Цезарь по латинице)

Мы ожидаем, что **внутри одной строки** ключ один и тот же для `email` и `Адрес`.
Поэтому выбираем ключ совместно: `score(address) + score(email)`.

In [10]:
from src.scoring import best_k_by_addr, best_k_joint

EMAIL_RE = re.compile(r"^[a-z0-9._%+-]+@[a-z0-9.-]+\.[a-z]{2,}$")
EMAIL_HINTS = [
    "gmail.com", "hotmail.com", "yandex.ru", "mail.ru", "outlook.com", "icloud.com",
    ".com", ".ru", ".net", ".org", ".biz", ".info"
]

# Сравнение на нескольких строках: ключ "по адресу" vs "совместный ключ"
tmp = df.sample(7, random_state=1).copy().reset_index(drop=True)
rows = []
for _, r in tmp.iterrows():
    k_addr, sc_addr = best_k_by_addr(r["Адрес"])
    k_joint, sc_sum, sc_a, sc_e = best_k_joint(r["email"], r["Адрес"])
    rows.append([k_addr, sc_addr, k_joint, sc_sum, sc_a, sc_e, r["email"], dec_email(r["email"], k_joint)])
pd.DataFrame(rows, columns=["k_addr", "addr_score", "k_joint", "sum_score", "addr_part", "email_part", "email_enc", "email_dec"])

,k_addr,addr_score,k_joint,sum_score,addr_part,email_part,email_enc,email_dec
0,4,11,4,19,11,8,erewxewme55@kqemp.gsq,anastasia55@gmail.com
1,19,13,19,22,13,9,ltgwkt.chaglmhg@zftbe.vhf,sandra.johnston@gmail.com
2,19,13,19,20,13,7,mkxot.mnkvhmmx@chgxl.hkz,treva.turcotte@jones.org
3,6,13,6,20,13,7,jogttg.qubgikq@qanoi.ius,dianna.kovacek@kuhic.com
4,8,13,8,21,13,8,zivlg38@ouiqt.kwu,randy38@gmail.com
5,3,13,3,21,13,8,plndbod56@krwpdlo.frp,mikayla56@hotmail.com
6,14,13,14,20,13,7,szgo.dcifcg@acgqwgyw.qca,elsa.pouros@mosciski.com


Применим совместный выбор ключа для всего датасета и построим деобезличенный результат:

In [11]:
keys = []
sum_scores = []
addr_part = []
email_part = []

for e, a in zip(df["email"].astype(str), df["Адрес"].astype(str)):
    k, ssum, sa, se = best_k_joint(e, a)
    keys.append(k)
    sum_scores.append(ssum)
    addr_part.append(sa)
    email_part.append(se)

out = df.copy()
out["Ключ_шифрования"] = keys
out["email_деобезличен"] = [dec_email(e, k) for e, k in zip(df["email"].astype(str), keys)]
out["Адрес_деобезличен"] = [dec_addr(a, k) for a, k in zip(df["Адрес"].astype(str), keys)]
out["score_addr"] = addr_part
out["score_email"] = email_part
out["score_total"] = sum_scores

out.head()

,Телефон,email,Адрес,Ключ_шифрования,email_деобезличен,Адрес_деобезличен,score_addr,score_email,score_total
0,47f0896ebfa7f70683e82c26bfdba33178d1d6a8,xgdlt@bpgfjpgsi.qxo,въ. Псчпщэьабявщбэяп Ычщэоьп у.5 щс.476,15,irowe@marquardt.biz,ул. Авиаконструктора Микояна д.5 кв.476,16,6,22
1,329510138dad2c8ae131b6c55a60ff4dbed1a43a,eyzuqky@nuzsgor.ius,Тлмкщужцфкуже щс. к.42 ри.379,6,ystokes@hotmail.com,Международная ул. д.42 кв.379,13,8,21
2,9da60383009e86d30c873f3f2b0a928a34575274,zivlg38@ouiqt.kwu,Ымияхиз ыу. м.44 тк.188,8,randy38@gmail.com,Удачная ул. д.44 кв.188,13,8,21
3,94fd3b56f2fec4c694465c84af9219e3c11d9ab2,erewxewme55@kqemp.gsq,уп. Хпджя и.34 ож.468,4,anastasia55@gmail.com,пл. Славы д.34 кв.468,11,8,19
4,2193b62bcfa2910745a5d80602d721dcc354361a,szgo.dcifcg@acgqwgyw.qca,Юьфтуяаруыяшон бщ. т.73 шр.174,14,elsa.pouros@mosciski.com,Рождественская ул. д.73 кв.174,13,7,20


Наглядный вывод (несколько строк):

In [12]:

out.sample(10, random_state=2)[["Телефон", "email", "email_деобезличен", "Адрес", "Адрес_деобезличен", "Ключ_шифрования"]]

,Телефон,email,email_деобезличен,Адрес,Адрес_деобезличен,Ключ_шифрования
9,e2edec54ca67ae51147b00148bd1c5cabe19c750,wkbskxxk17@cmrknox.ybq,marianna17@schaden.org,Ыкмёхфтчыфту щъ. о.17 фм.496,Савёлкинский пр. д.17 кв.496,10
4,2193b62bcfa2910745a5d80602d721dcc354361a,szgo.dcifcg@acgqwgyw.qca,elsa.pouros@mosciski.com,Юьфтуяаруыяшон бщ. т.73 шр.174,Рождественская ул. д.73 кв.174,14
14,56bcbc42c33836c8d20372f0bb46116ceeaea136,plndbod56@krwpdlo.frp,mikayla56@hotmail.com,2-в Оифрсувзфнгв цо. з.88 не.299,2-я Леснорядская ул. д.88 кв.299,3
0,47f0896ebfa7f70683e82c26bfdba33178d1d6a8,xgdlt@bpgfjpgsi.qxo,irowe@marquardt.biz,въ. Псчпщэьабявщбэяп Ычщэоьп у.5 щс.476,ул. Авиаконструктора Микояна д.5 кв.476,15
10,8b95d0b94ec883e2280ea7598deb50428d378c6e,jogttg.qubgikq@qanoi.ius,dianna.kovacek@kuhic.com,щс. Торсщыф-Тжрсже к.33 ри.137,ул. Миклухо-Маклая д.33 кв.137,6
5,9fd31a1db8d4177b8f624f7642634c515cba9bfd,dbowlvki.toppoboi@rydwksv.myw,tremblay.jefferey@hotmail.com,эх. Йъапмыфтп Щшхйче-2 о.67 фм.470,ул. Ярцевские Поляны-2 д.67 кв.470,10
3,94fd3b56f2fec4c694465c84af9219e3c11d9ab2,erewxewme55@kqemp.gsq,anastasia55@gmail.com,уп. Хпджя и.34 ож.468,пл. Славы д.34 кв.468,4
1,329510138dad2c8ae131b6c55a60ff4dbed1a43a,eyzuqky@nuzsgor.ius,ystokes@hotmail.com,Тлмкщужцфкуже щс. к.42 ри.379,Международная ул. д.42 кв.379,6
12,bd364baa4de7cf46746b1572cafb819fc663bf65,iygajvea.nqjkhbozkppen@oydiepp.yki,mckenzie.runolfsdottir@schmitt.com,йб. Иювюжхэышц ъ.63 аш.19,ул. Тимирязева д.63 кв.19,22
7,ad71f14a8bc97e653a4648423d1028a0e4036d7c,lhlk.ljbo@elqjxfi.zlj,okon.omer@hotmail.com,Юдчгьдибчц кв. ы.6 бщ.271,Знаменская ул. д.6 кв.271,23


## 5) Анализ телефонных номеров

1) Длины популярных хэшей в hex:
- MD5 → 32 символа
- SHA‑1 → 40 символов
- SHA‑256 → 64 символа
- SHA‑512 → 128 символов

2) Проверяем, что значения состоят из hex-символов.

Если все значения — **40 hex символов**, то это наиболее характерно для **SHA‑1**.

In [13]:
phones = df["Телефон"].astype(str).str.strip()

lengths = phones.map(len).value_counts().sort_index()
lengths


Телефон
40    19
Name: count, dtype: int64

### 5.1) Демонстрация: SHA‑1 действительно даёт 40 hex символов

In [14]:
import hashlib

example = "89991234567"
h = hashlib.sha1(example.encode("utf-8")).hexdigest()
h, len(h)

('5af0900c49233000e4e7a64295faf2231b77d2d6', 40)

### 5.2) Деобезличивание с помощью hashcat

Hashcat — программа для восстановления паролей путём перебора хэшей. Она позволяет проводить "brute-force" атаки (перебором на основе заранее выбранной маски). В качестве маски используем шаблон из 11 цифр: `?d?d?d?d?d?d?d?d?d?d?d`

In [ ]:
def identify(hashes: List[str]) -> None:
    with open('hashes.txt', 'w') as f:
        for HASH in hashes:
            f.write(HASH + "\n")
    os.system("hashcat.exe -a 3 -m 100 -o output.txt hashes.txt ?d?d?d?d?d?d?d?d?d?d?d")

identify(phones.tolist()) 

In [23]:
with open("output.txt", "r") as f:
    lines = f.readlines()
hash_to_phone = {}
for line in lines:
    parts = line.strip().split(":")
    if len(parts) == 2:
        hash_val, phone = parts
        hash_to_phone[hash_val] = phone

hash_to_phone


{'ed3192791d0ae8396c875b9b1e6e189c068446c1': '89649854900',
 'df4df66d1f14191b8367166633f18339893cf998': '89998643700',
 '329510138dad2c8ae131b6c55a60ff4dbed1a43a': '89019200378',
 '56bcbc42c33836c8d20372f0bb46116ceeaea136': '89196758378',
 '9fd31a1db8d4177b8f624f7642634c515cba9bfd': '89868693101',
 'ad71f14a8bc97e653a4648423d1028a0e4036d7c': '89589567019',
 '27810d28306b0a0765b9b7a5bfbb91ba00a83f04': '89299488080',
 '8b95d0b94ec883e2280ea7598deb50428d378c6e': '89108689925',
 '44c58c09f65953cb2a923392b0de62a0c1c3e5f2': '89367766379',
 '13f138e5af9ce59d79b0c2635bf53ce4df3fc025': '89998286102',
 '9da60383009e86d30c873f3f2b0a928a34575274': '89366762354',
 '94fd3b56f2fec4c694465c84af9219e3c11d9ab2': '89106432726',
 'bd364baa4de7cf46746b1572cafb819fc663bf65': '89017765368',
 '2193b62bcfa2910745a5d80602d721dcc354361a': '89676986297',
 'e2edec54ca67ae51147b00148bd1c5cabe19c750': '89198688497',
 'e742b4c0758720cd6e76e87d36dfd58565c18702': '89018960148',
 '8d99c58c8f9e4a422f0a74b5528abdcfee6289

In [28]:
out['Телефон_деобезличен'] = out['Телефон'].astype(str).map(hash_to_phone)

out.head()

,Телефон,email,Адрес,Ключ_шифрования,email_деобезличен,Адрес_деобезличен,score_addr,score_email,score_total,Телефон_деобезличен
0,47f0896ebfa7f70683e82c26bfdba33178d1d6a8,xgdlt@bpgfjpgsi.qxo,въ. Псчпщэьабявщбэяп Ычщэоьп у.5 щс.476,15,irowe@marquardt.biz,ул. Авиаконструктора Микояна д.5 кв.476,16,6,22,89996999664
1,329510138dad2c8ae131b6c55a60ff4dbed1a43a,eyzuqky@nuzsgor.ius,Тлмкщужцфкуже щс. к.42 ри.379,6,ystokes@hotmail.com,Международная ул. д.42 кв.379,13,8,21,89019200378
2,9da60383009e86d30c873f3f2b0a928a34575274,zivlg38@ouiqt.kwu,Ымияхиз ыу. м.44 тк.188,8,randy38@gmail.com,Удачная ул. д.44 кв.188,13,8,21,89366762354
3,94fd3b56f2fec4c694465c84af9219e3c11d9ab2,erewxewme55@kqemp.gsq,уп. Хпджя и.34 ож.468,4,anastasia55@gmail.com,пл. Славы д.34 кв.468,11,8,19,89106432726
4,2193b62bcfa2910745a5d80602d721dcc354361a,szgo.dcifcg@acgqwgyw.qca,Юьфтуяаруыяшон бщ. т.73 шр.174,14,elsa.pouros@mosciski.com,Рождественская ул. д.73 кв.174,13,7,20,89676986297


In [31]:
out[['Телефон_деобезличен', 'email_деобезличен', 'Адрес_деобезличен', 'Ключ_шифрования']].to_excel('Деобезличенные_данные+ключ_шифрования.xlsx')

## 6) Итог

Мы получили:
- Для `Адреса` и `email`: обезличивание = **шифр Цезаря (циклический сдвиг)**, ключ разный на каждую строку.
- Для `Телефона`: формат соответствует **SHA‑1**, восстанавливается с помощью brute-force атаки (hashcat).